In [2]:
import pandas as pd
df=pd.read_csv("tweet_emotions2.csv",encoding="latin")
df

,sentiment,content
0,empty,@tiffanylue i know i was listenin to bad habi...
1,sadness,Layin n bed with a headache ughhhh...waitin o...
2,sadness,Funeral ceremony...gloomy friday...
3,enthusiasm,wants to hang out with friends SOON!
4,neutral,@dannycastillo We want to trade with someone w...
...,...,...
74787,surprise,@MichelGW have you gift! Hope you like it! It'...
74788,joy,The world didnt give it to me..so the world MO...
74789,anger,A man robbed me today .
74790,fear,"Youu call it JEALOUSY, I call it of #Losing YO..."


In [3]:
df['sentiment'].unique()

array(['empty', 'sadness', 'enthusiasm', 'neutral', 'worry', 'surprise',
       'love', 'fun', 'hate', 'happiness', 'boredom', 'relief', 'anger',
       'joy', 'fear', 'shame', 'disgust'], dtype=object)

In [5]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM, Dense, Embedding, SpatialDropout1D
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, accuracy_score

# Load the dataset
data = pd.read_csv("tweet_emotions2.csv",encoding="latin")

# Drop rows with missing values
data.dropna(subset=['content', 'sentiment'], inplace=True)

# Prepare the data
text = data['content'].astype(str).values
labels = data['sentiment'].values

# Encode labels to numeric values
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels)

# Tokenize the text
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text)
vocab_size = len(tokenizer.word_index) + 1
sequences = tokenizer.texts_to_sequences(text)
max_sequence_length = max([len(seq) for seq in sequences])
padded_sequences = pad_sequences(sequences, maxlen=max_sequence_length)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(padded_sequences, labels, test_size=0.2, random_state=42)

# Define the LSTM model
model = Sequential()
model.add(Embedding(vocab_size, 128, input_length=max_sequence_length))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(18, activation='softmax'))

# Compile the model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

# Make predictions
y_pred_prob = model.predict(X_test)
y_pred = np.argmax(y_pred_prob, axis=1)


accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)


print("Accuracy:", accuracy)
print("Classification Report:")
print(report)


Epoch 1/10
1870/1870 [==============================] - 670s 358ms/step - loss: 2.0604 - accuracy: 0.3179 - val_loss: 1.8389 - val_accuracy: 0.4051
Epoch 2/10
1870/1870 [==============================] - 1102s 590ms/step - loss: 1.6204 - accuracy: 0.4759 - val_loss: 1.7896 - val_accuracy: 0.4220
Epoch 3/10
1870/1870 [==============================] - 842s 450ms/step - loss: 1.2802 - accuracy: 0.5956 - val_loss: 1.8966 - val_accuracy: 0.4128
Epoch 4/10
1870/1870 [==============================] - 683s 365ms/step - loss: 0.9800 - accuracy: 0.6944 - val_loss: 2.0598 - val_accuracy: 0.4050
Epoch 5/10
1870/1870 [==============================] - 673s 360ms/step - loss: 0.7501 - accuracy: 0.7676 - val_loss: 2.2791 - val_accuracy: 0.3917
Epoch 6/10
1870/1870 [==============================] - 666s 356ms/step - loss: 0.6012 - accuracy: 0.8112 - val_loss: 2.4750 - val_accuracy: 0.3899
Epoch 7/10
1870/1870 [==============================] - 652s 348ms/step - loss: 0.5015 - accuracy: 0.8423 - val